In [ ]:
import os, sys, h5py, json, time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    from tqdm import tqdm
    tqdm.pandas() # suppress annoying FutureWarning

import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama import DataGroup, lh5
from pygama.dsp.dsp_optimize import *
from pygama.dsp.WaveformBrowser import WaveformBrowser as wfb

In [ ]:
query = 'cycle > 2185 and cycle < 2188'

# set raw energy estimator
etype = 'energy'
tb_in = 'ORSIS3302DecoderForEnergy/raw'
wf_in = 'ORSIS3302DecoderForEnergy/raw/waveform'
xlo, xhi, xpb = 0, 4e6, 10000

nwfs = 100
norm = True
tp_align = 0.01
n_pre, n_post = 3500, 3500

In [ ]:
dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
dg.fileDB.query(query, inplace=True)

# limit this test to just the first file in the list
raw_files = dg.lh5_dir + dg.fileDB['raw_path'] + '/' + dg.fileDB['raw_file']
raw_file = raw_files.tolist()[0]

# load the energy array to make a cut
ene_vals = lh5.load_nda(raw_file, ['energy'], tb_in)['energy']

In [ ]:
# pull a limited number of waveforms passing a cut on energy

ix_ene = np.where((ene_vals > 1e6) & (ene_vals < 1e7))[0][:nwfs]

sto = lh5.Store()
tb_raw, n_rows = sto.read_object(tb_in, raw_file, idx=ix_ene)
wfs = tb_raw['waveform']['values'].nda

print(ix_ene)
print(n_rows)
print(wfs.shape)

In [ ]:
# baseline subtraction
bl_means = wfs[:,:3500].mean(axis=1)
wf_blsub = (wfs.transpose() - bl_means).transpose()

# time alignment
ts = np.arange(0, wf_blsub.shape[1], 1)

wf_maxes = np.amax(wf_blsub, axis=1)
timepoints = np.argmax(wf_blsub >= wf_maxes[:, None]*tp_align, axis=1)

# idxs: [n_rows, (n_pre+n_post samples)]
wf_idxs = np.zeros([wf_blsub.shape[0], n_pre + n_post], dtype=int)
row_idxs = np.zeros_like(wf_idxs)
for i, tp in enumerate(timepoints):
    wf_idxs[i, :] = np.arange(tp - n_pre, tp + n_post, 1)
    row_idxs[i, :] = i

wf_win = wf_blsub[row_idxs, wf_idxs]

# normalize to pulse height maximum
wf_norm = np.divide(wf_win, wf_maxes[:,None])

In [ ]:
%matplotlib widget

ts = np.arange(0, wf_norm.shape[1], 1)

for iwf in range(0, wf_norm.shape[0]):
    wf = wf_norm[iwf,:]
    plt.plot(ts, wf, alpha=0.5)
    
plt.xlim(3400, 3600)
plt.show()